<h1><b>Normal level</b></h1>

Константы, используемые в работе

In [156]:
N = 501780 % 20

m = 4 + (N % 5)
n = 2 + (N % 5)
k = 1 + (N % 7)

n1 = N % 6
n2 = (N + 1) % 6
n3 = (N + 2) % 6

p = 29
s = 5
r = 59
t = 9

Импорт всех необходимых библиотек

In [157]:
from sympy.combinatorics import Permutation, PermutationGroup
from sympy.combinatorics.named_groups import SymmetricGroup
import itertools
import math
import random
from math import gcd
from sympy import symbols, Poly, GF, factor, invert
from sympy.polys.polytools import gcdex
from itertools import product

<h2>Task 1</h2>

Найдите все подгруппы симметрической группы S(m). Выведите их количество и однуслучайную подгруппу. Для подгруппы с индексом N mod (числоподгрупп) постройте левые и правые смежные классы, определите её индекс и проверьте, является ли она нормальной.

In [ ]:
def is_subgroup(G, elements_set):
    """Проверяет, является ли множество элементов подгруппой"""
    if not elements_set:
        return False
    
    identity = Permutation(G.degree - 1)
    
    # Проверка наличия нейтрального элемента
    if identity not in elements_set:
        return False
    
    # Проверка замкнутости относительно умножения
    elements_list = list(elements_set)
    for a in elements_list:
        for b in elements_list:
            if a * b not in elements_set:
                return False
    
    # Проверка наличия обратных элементов
    for a in elements_list:
        if a ** -1 not in elements_set:
            return False
    
    return True


def generate_subgroup_from_elements(G, elements):
    """Генерирует подгруппу из элементов и проверяет аксиомы"""
    identity = Permutation(G.degree - 1)
    elements_set = set(elements)
    elements_set.add(identity)  # Добавляем нейтральный элемент
    
    # Проверяем аксиомы группы
    if not is_subgroup(G, elements_set):
        return None
    
    # Создаем группу из этих элементов
    return PermutationGroup(*[e for e in elements_set ])


def find_all_subgroups(N: int) -> list:
    """Находит все подгруппы S(m)"""
    S = SymmetricGroup(N)
    all_elements = list(S.generate())
    identity = Permutation(N - 1)
    
    subgroups = []
    
    # Все возможные порядки подгрупп (по теореме Лагранжа)
    possible_orders = []
    lim = math.factorial(N)
    for i in range(1, lim + 1):
        if lim % i == 0 and i not in possible_orders:
            possible_orders.append(i)
    
    # Перебираем все возможные размеры подмножеств
    for size in possible_orders:
        if size == 1:
            # Тривиальная подгруппа
            trivial = PermutationGroup(identity)
            subgroups.append(trivial)
            continue
        
        if size == possible_orders[-1]:
            # Вся группа
            subgroups.append(S)
            continue
        
        # Перебираем комбинации элементов
        for comb in itertools.combinations(all_elements, size - 1):  # -1 потому что добавляем identity
            # Пропускаем комбинации без identity (он будет добавлен)
            if identity in comb:
                continue
            
            # Пробуем создать подгруппу
            subgroup = generate_subgroup_from_elements(S, comb)
            
            if subgroup is not None and subgroup.order() == size:
                # Проверяем, не добавили ли уже эту подгруппу
                is_duplicate = False
                for existing in subgroups:
                    # Проверка на равенство подгрупп
                    if existing == subgroup:
                        is_duplicate = True
                        break
                if not is_duplicate:
                    subgroups.append(subgroup)
                    # print('found')
    
    return subgroups


def analyze_cosets(G, H):
    """Строит смежные классы"""
    elements_G = list(G.generate())
    elements_H = list(H.generate())
    
    left_cosets = []
    used = set()
    for g in elements_G:
        if g not in used:
            coset = {g * h for h in elements_H}
            left_cosets.append(coset)
            used.update(coset)
    
    right_cosets = []
    used = set()
    for g in elements_G:
        if g not in used:
            coset = {h * g for h in elements_H}
            right_cosets.append(coset)
            used.update(coset)
    
    return left_cosets, right_cosets


def subgroups_of_Sm(N: int, m) -> dict:
    """Анализ симметрической группы S(m) с использованием sympy"""
    
    # Создаем симметрическую группу
    G = SymmetricGroup(m)
    
    # Генерируем все подгруппы
    subgroups = find_all_subgroups(m)
    
    
    # Выбираем случайную подгруппу
    random_subgroup = random.choice(subgroups)
    
    # Выбираем подгруппу по индексу N mod (число подгрупп)
    index = N % len(subgroups)
    H = subgroups[index]
    
    # Находим смежные классы
    left_cosets, right_cosets = analyze_cosets(G, H)
    
    # Индекс подгруппы
    index_H = len(left_cosets)
    
    # Проверяем нормальность
    if left_cosets == right_cosets:
        is_normal = True
    else:
        is_normal = False
    
    return {
        'group': G,
        'total_subgroups': len(subgroups),
        'subgroups': subgroups,
        'random_subgroup': random_subgroup,
        'selected_subgroup': H,
        'index': index_H,
        'is_normal': is_normal,
        'left_cosets': left_cosets,
        'right_cosets': right_cosets
    }


result = subgroups_of_Sm(N, m)

print(f"Количество найденных подгрупп: {result['total_subgroups']}")
print(f"Случайна подргуппа: {result['random_subgroup']}")
print(f"Выбранная подруппа (индекс {result['index']}): {result['selected_subgroup']}")

print(f"Левые смежные классы (gH):")
for i, coset in enumerate(result['left_cosets']):
        print(f"Смежный класс {i}: {len(coset)} элементов")
        if len(coset) <= 10:  # Показываем содержимое только для маленьких классов
            print(f"  {coset}")

print(f"Правые смежные классы (Hg):")
for i, coset in enumerate(result['right_cosets']):
    print(f"Смежный класс {i}: {len(coset)} элементов")
    if len(coset) <= 10:
        print(f"  {coset}")

print(f"\nЯвляется ли подгруппа нормальной: {result['is_normal']}")


Количество найденных подгрупп: 30
Случайна подргуппа: PermutationGroup([
    (0 3)(1 2),
    (0 1 3 2),
    (0 2 3 1)])
Выбранная подруппа (индекс 24): PermutationGroup([
    (3)])
Левые смежные классы (gH):
Смежный класс 0: 1 элементов
  {Permutation(3)}
Смежный класс 1: 1 элементов
  {Permutation(0, 1, 2, 3)}
Смежный класс 2: 1 элементов
  {Permutation(0, 2)(1, 3)}
Смежный класс 3: 1 элементов
  {Permutation(0, 3)}
Смежный класс 4: 1 элементов
  {Permutation(1, 2, 3)}
Смежный класс 5: 1 элементов
  {Permutation(0, 1, 3, 2)}
Смежный класс 6: 1 элементов
  {Permutation(3)(0, 2, 1)}
Смежный класс 7: 1 элементов
  {Permutation(0, 3, 1, 2)}
Смежный класс 8: 1 элементов
  {Permutation(1, 3, 2)}
Смежный класс 9: 1 элементов
  {Permutation(3)(0, 1)}
Смежный класс 10: 1 элементов
  {Permutation(0, 2, 3)}
Смежный класс 11: 1 элементов
  {Permutation(0, 3, 2, 1)}
Смежный класс 12: 1 элементов
  {Permutation(2, 3)}
Смежный класс 13: 1 элементов
  {Permutation(3)(0, 1, 2)}
Смежный класс 14: 1 эле

<h2>Task 2</h2>

В группе S(m) возьмите элемент с индексом N % |S(m)|. Найдите порядки элементов g^n1, g^n2, g^n3 и порядки циклических групп, ими пораждаемых.

In [ ]:
def find_order(G, g):
    """Считает порядок элемента группы перестановок"""
    identity = Permutation(G.degree - 1)
    order = 1
    while g != identity:
        g *= g
        order += 1
    
    return order
    

def element_powers_in_Sm(N: int, m: int, n1: int, n2: int, n3: int, result) -> dict:
    S_m = SymmetricGroup(m)
    element = S_m.elements[N % S_m.order()]
    g1 = element ** n1
    g2 = element ** n2
    g3 = element ** n3
    orders = {
        'element': element,
        'g1': g1,
        'g2': g2,
        'g3': g3,
        'g^n1': find_order(S_m, g1),
        'g^n2': find_order(S_m, g2),
        'g^n3': find_order(S_m, g3),
        'orders': set()
    }
    subgroups = [PermutationGroup([g1]), PermutationGroup([g2]), PermutationGroup([g3]),
                 PermutationGroup([g1, g2]), PermutationGroup([g1, g3]), PermutationGroup([g2, g3]),
                 PermutationGroup([g1, g2, g3])]
    for subgroup in subgroups:
        if subgroup in result['subgroups']:
            orders['orders'].add(len(subgroup))
            continue
    return orders


powers = element_powers_in_Sm(N, m, n1, n2, n3, result)
for key, word in powers.items():
    if '^' in key:
        print(f'Порядок елемента {key} (равного {powers[key[0] + key[-1]]}): {word}')
    elif key == 'orders':
        print(f'Порядки циклических подгрупп, пораждаемых этими элементами: {word}')

Порядок елемента g^n1 (равного (3)): 1
Порядок елемента g^n2 (равного (3)): 1
Порядок елемента g^n3 (равного (3)): 1
Порядки циклических подгрупп, пораждаемых этими элементами: {1}


В функции перебираем все возможные комбинации g_i, смотрим, являются ли они подгруппами группы перестановок и если да, то считаем их порядок

<h2>Task 3</h2>

В группе Sm найдите все решения уравление s^n = (1, 2, 3 ... m - 1). Вывести количество решений и три случайных решения. Опишите, что у них общего.

In [96]:
def compoze_n(element, n: int):
    res = element
    for _ in range(n - 1):
        res *= element
    return res


def solve_sigma_power_eq(N: int, m: int, n: int) -> dict:
    S = SymmetricGroup(m)
    elements = list(S.generate())

    result = {}
    target = Permutation([range(m - 1)])
    for element in elements:
        if compoze_n(element, n) == target:
            result[f'{element}'] = 'solve'
    
    return result

task3 = solve_sigma_power_eq(N, m, n)
if task3:
    for key, word in task3.items():
        print(f'{key} является решением уравления')
else:
    print('Решений нет')

Решений нет


В группе перестановок 4 нет решений уравнения s^2 = (0, 1, 2)

<h2>Task 4</h2>

В циклической группе порядка m найдите: все элементы g, такие что g^k = e, все элементы порядка k.

In [ ]:
def elements_of_order_k_in_cyclic_group(N: int, m: int, k: int) -> dict:
    results = {
        'elements_gk': [],  # элемент в степени k равен нейтральному
        'elements_of_order_k': [],  # элементы которые зацикливают
    }

    G = range(m)

    identiti = 0

    for g in G:
        if (g * k) % m == identiti:  # Степень дает нейтральный
            results['elements_gk'].append(g)
        if g * k == m or g * k == 0:  # Степень дает 0 или m
            results['elements_of_order_k'].append(g)
    
    return results


results = elements_of_order_k_in_cyclic_group(N, m, k)

print(f"\nk = {k}:")
print(f"Элементы, удовлетворяющие g^{k} = e: {results['elements_gk']}")

print(f"Элементы порядка {k}: {results['elements_of_order_k']}")



k = 1:
Элементы, удовлетворяющие g^1 = e: [0]
Элементы порядка 1: [0]


Важное различие, g^k=e не значит, что k является порядком элемента, порядок у элемента один, а решений уравнения g^k=e бесконечно много

<h2>Task 5</h2>

Найти все подгруппы мультипликативной группы Zm.

In [ ]:
def subgroups_of_Zn_star(N: int, m: int) -> list:
    Zm = [a for a in range(1, m) if gcd(a, m) == 1]
    
    subgroups = []
    subgroups.append({1})  # Тривиальная
    
    # Для каждого элемента находим порожденную им циклическую подгруппу
    for a in Zm:
        if a == 1:
            continue
            
        # Находим циклическую подгруппу, порожденную a
        subgroup = set()
        current = a
        
        while current not in subgroup:
            subgroup.add(current)
            current = (current * a) % n
            if current == 1:
                break
        
        # Проверяем, не добавили ли уже эту подгруппу
        if subgroup not in subgroups:
            subgroups.append(subgroup)
    
    # Добавляем саму группу
    if set(Zm) not in subgroups:
        subgroups.append(set(Zm))
    
    # Сортируем по размеру
    subgroups.sort(key=len)
    
    return subgroups


result = subgroups_of_Zn_star(N, m)

print(f"Z*_{m} = {[a for a in range(1, m) if gcd(a, m) == 1]}")
print(f"Все подгруппы Z*_{m}:")
for i, subgroup in enumerate(result):
    print(f"Подгруппа {i+1}: {sorted(subgroup)} (порядок {len(subgroup)})")

Z*_4 = [1, 3]
Все подгруппы Z*_4:
Подгруппа 1: [1] (порядок 1)
Подгруппа 2: [3] (порядок 1)
Подгруппа 3: [1, 3] (порядок 2)


Мультипликативной группой по умножению называется такая группа обратимых элементов кольца Zm, относительно операции умножения, что gcd(a, m) = 1, где а - элемент кольца

<h2>Task 6</h2>

Пусть s ∈ Zp∗. Найдите порядок элемента s^r в Zp∗.

s^r = e

In [158]:
def order_of_sr(p: int, s: int, r: int) -> int:
    # Проверяем, что s принадлежит Z_p^*
    if gcd(s, p) != 1:
        raise ValueError("s должен быть взаимно прост с p")
    
    # Находим порядок элемента s в Z_p^*
    order_s = 1
    current = s
    while current != 1:
        current = (current * s) % p
        order_s += 1
    
    return order_s


task6 = order_of_sr(p, s, r)
print(f"Порядок элемента {s}^{r} в Z_{p}^*: {task6}")

Порядок элемента 5^59 в Z_29^*: 14


<h2>Task 7</h2>

Найдите порядок элемента t вгруппе Zp∗. Является ли t образующим (примитивным корнем) ?

Образующий элемент - элемент, через степень которого можно выразить любой элемент группы

In [ ]:
def order_and_primitivity_of_t(p: int, t: int) -> dict:
    # Проверяем, что t принадлежит Z_p^*
    if gcd(t, p) != 1:
        raise ValueError("t должен быть взаимно прост с p")
    
    # Находим порядок элемента t
    order = 1
    current = t
    while current != 1:
        current = (current * t) % p
        order += 1
    
    # t - образующий, если его порядок равен p-1
    is_primitive = (order == p - 1)
    
    return {
        'order': order,
        'is_primitive': is_primitive,
        'group_order': p - 1
    }


task7 = order_and_primitivity_of_t(p, t)
print(f"t = {t} в Z_{p}*:")
print(f"Порядок: {task7['order']}")
print(f"Образующий: {'Да' if task7['is_primitive'] else 'Нет'}")
print(f"Порядок группы: {task7['group_order']}")


t = 9 в Z_29^*:
Порядок: 14
Образующий: Нет
Порядок группы: 28


<h2>Task 8</h2>

Найдите все образующие (примитивные корни) циклическойгруппы Zm∗.

NB Мультипликативная группа Zm и циклическая группа Zm это то же самое, но в циклической обязательно существует пораждающий элемент.

In [ ]:
def generators_of_Zm_star(m: int) -> list:
    # Строим группу Z_m^*
    Zm_star = [a for a in range(1, m) if gcd(a, m) == 1]
    group_order = len(Zm_star)
    
    generators = []
    
    # Проверяем каждый элемент на то, является ли он образующим
    for a in Zm_star:
        # Находим порядок элемента a
        order = 1
        current = a
        while current != 1:
            current = (current * a) % m
            order += 1
        
        # Элемент - образующий, если его порядок равен порядку группы
        if order == group_order:
            generators.append(a)
    
    return generators


result = generators_of_Zm_star(m)

print(f"Z_{m}^* = {[a for a in range(1, m) if gcd(a, m) == 1]}")
print(f"Порядок группы: {len([a for a in range(1, m) if gcd(a, m) == 1])}")
print(f"Примитивные корни: {result}")
print(f"Количество примитивных корней: {len(result)}")

Z_4^* = [1, 3]
Порядок группы: 2
Примитивные корни: [3]
Количество примитивных корней: 1


<h2>Task 9</h2>

В аддитивной группе Zm найдите циклическую подгруппу, порождённую элементом t mod m. Определите её порядок и все порождающие элементы. Опишите связь с исходным t.

In [153]:
def cyclic_subgroup_in_Zm_additive(m: int, t: int) -> dict:
    # Аддитивная группа Z_m = {0, 1, 2, ..., m-1}
    
    # Находим циклическую подгруппу, порожденную t
    subgroup = set()
    current = t % m
    
    while current not in subgroup:
        subgroup.add(current)
        current = (current + t) % m
    
    # Порядок подгруппы
    order = len(subgroup)
    
    # Находим все порождающие элементы этой подгруппы
    generators = []
    for element in subgroup:
        if element != 0:  # 0 никогда не порождает нетривиальную подгруппу
            # Проверяем, порождает ли element ту же подгруппу
            test_subgroup = set()
            test_current = element
            
            while test_current not in test_subgroup:
                test_subgroup.add(test_current)
                test_current = (test_current + element) % m
            
            if test_subgroup == subgroup:
                generators.append(element)
    
    return {
        'subgroup': sorted(subgroup),
        'order': order,
        'generators': sorted(generators),
        'gcd_value': gcd(t, m)
    }


result = cyclic_subgroup_in_Zm_additive(m, t)

print(f"Аддитивная группа Z_{m}")
print(f"Циклическая подгруппа, порожденная элементом {t % m}:")
print(f"Подгруппа: {result['subgroup']}")
print(f"Порядок подгруппы: {result['order']}")
print(f"Порождающие элементы: {result['generators']}")
print(f"НОД({t}, {m}) = {result['gcd_value']}")

Аддитивная группа Z_4
Циклическая подгруппа, порожденная элементом 1:
Подгруппа: [0, 1, 2, 3]
Порядок подгруппы: 4
Порождающие элементы: [1, 3]
НОД(9, 4) = 1


С t связан порядок группы
Те порядок группы = m / gcd(t, m)
В данном случае 4 / 1 = 4

<h2>Task 10</h2>

В мультипликативной группе Zm∗ найдите циклическую подгруппу, порождённую элементом t mod m. Определите, какой циклической подгруппе в симметрической группе Sd (где d — порядок подгруппы) она изоморфна.

In [154]:
def isomorphism_of_cyclic_subgroup_Zm_star(m: int, t: int) -> dict:
    if gcd(t, m) != 1:
        raise ValueError("t должен быть взаимно прост с m")
    
    # Находим циклическую подгруппу, порожденную t
    subgroup = set()
    current = t % m
    
    while current not in subgroup:
        subgroup.add(current)
        current = (current * t) % m
    
    # Порядок подгруппы
    order = len(subgroup)
    subgroup_sorted = sorted(subgroup)
    
    # Правильный изоморфизм:
    # Циклическая подгруппа порядка d изоморфна аддитивной группе Z_d
    # И изоморфна циклической ПОДГРУППЕ S_d, порожденной d-циклом
    
    return {
        'subgroup': subgroup_sorted,
        'order': order
    }


result = isomorphism_of_cyclic_subgroup_Zm_star(m, t)

print(f"Мультипликативная группа Z*_{m}")
print(f"Циклическая подгруппа, порожденная элементом {t % m}:")
print(f"Подгруппа: {result['subgroup']}")
print(f"Порядок подгруппы: {result['order']}")

Мультипликативная группа Z*_4
Циклическая подгруппа, порожденная элементом 1:
Подгруппа: [1]
Порядок подгруппы: 1


Изоморфна циклической подгруппе ((1)), тк в данном случае порядок группы равен 1, те можем построить f: 1(Z(t % m)) -> 1(S_1)

Умножение в Zm* изоморфно сложению в Z_d изоморфно композиции циклов в S_d

<h2>Task 11</h2>

Найти корни полиномов.

In [ ]:
def find_polynomial_roots_with_coefficients(N):
    x = symbols('x')
    # Вычисляем коэффициенты
    a_coeffs = [(i + N) % 4 for i in range(9)]
    b_coeffs = [(j + N) % 7 for j in range(7)]
    
    roots_F4 = Poly(x ** 9 + sum(a_coeffs[i] * x ** i for i in range(9)), x, domain=GF(4)).nroots()
    
    roots_F7 = Poly(sum(b_coeffs[i] * x ** i for i in range(7)), x, domain=GF(7)).nroots()
    
    return {
        'N': N,
        'a_coeffs': a_coeffs,
        'b_coeffs': b_coeffs,
        'roots_F4': roots_F4,
        'roots_F7': roots_F7
    }


result = find_polynomial_roots_with_coefficients(N)
for key, word in result.items():
    print(key, word)


Коэффициенты a_i = (i + 0) mod 4: [0, 1, 2, 3, 0, 1, 2, 3, 0]
Коэффициенты b_j = (j + 0) mod 7: [0, 1, 2, 3, 4, 5, 6]
f(x) = x⁹ + 0x⁸ + 3x⁷ + 2x⁶ + 1x⁵ + 0x⁴ + 3x³ + 2x² + 1x + 0
Корни в F₄: [-1.50276079771441, -0.414512842008724, 0, -0.628054601478571 - 0.727637196105672*I, -0.628054601478571 + 0.727637196105672*I, 0.686452987654979 - 0.972939545370781*I, 0.686452987654979 + 0.972939545370781*I, 0.90023843368516 - 0.644273025780718*I, 0.90023843368516 + 0.644273025780718*I]
N 0
a_coeffs [0, 1, 2, 3, 0, 1, 2, 3, 0]
b_coeffs [0, 1, 2, 3, 4, 5, 6]
roots_F4 [-1.50276079771441, -0.414512842008724, 0, -0.628054601478571 - 0.727637196105672*I, -0.628054601478571 + 0.727637196105672*I, 0.686452987654979 - 0.972939545370781*I, 0.686452987654979 + 0.972939545370781*I, 0.90023843368516 - 0.644273025780718*I, 0.90023843368516 + 0.644273025780718*I]
roots_F7 [0, 1.00000000000000, -1.22322069667983 - 1.70974018289378*I, -1.22322069667983 + 1.70974018289378*I, -0.276779303320166 - 0.386864527361488*

Считаем коэффиценты по условию и ищем корни при помощи sympy

<h2>Task 12</h2>

Исследуйте полиномы на приводимость. Приводимые полиномы разложите на неприводимые множители.

In [142]:
def factorize_polynomials_over_fields(N: int) -> dict:
    """
    Проверяет два полинома на приводимость и разлагает приводимые.
    """

    x = symbols('x')

    coeffs_c = [(k + N) % 5 for k in range(5)]
    coeffs_d = [(l + N) % 9 for l in range(4)]

    f1 = Poly(x**5 + sum(coeffs_c[i] * x**i for i in range(5)), x, domain=GF(5))
    f2 = Poly(x**4 + sum(coeffs_d[i] * x**i for i in range(4)), x, domain=GF(9))

    return {'f1_factorized': factor(f1), 'f2_factorized': factor(f2)}


task12 = factorize_polynomials_over_fields(N)
print("Разложение f(x) над GF(5):", task12['f1_factorized'])
print("Разложение f(x) над GF(9):", task12['f2_factorized'])


Разложение f(x) над GF(5): Poly(x**5 - x**4 - 2*x**3 + 2*x**2 + x, x, domain='ZZ')
Разложение f(x) над GF(9): Poly(x**4 + 3*x**3 + 2*x**2 + x, x, domain='ZZ')


Вывод: полиномы неприводымые

<h2>Task 13</h2>

Найти gcd(f, g) и его линейное представление.

In [ ]:
def gcd_and_linear_representation(N: int) -> dict:
    r_coeffs = [(m + N) % 11 for m in range(8)]
    s_coeffs = [(t + N) % 11 for t in range(4)]

    x = symbols('x')
    f = Poly(sum(r_coeffs[i] * x**i for i in range(8)), x, domain=GF(11))
    g = Poly(sum(s_coeffs[i] * x**i for i in range(4)), x, domain=GF(11))

    f, g, d = gcdex(f, g)
    return {'gcd': d, 'f': f, 'g': g}


task13 = gcd_and_linear_representation(N)
print("НОД полиномов над GF(11):", task13['gcd'])
print("f(x):", task13['f'])
print("g(x):", task13['g'])
print(f'gcd(f, g) = {task13['gcd']} * {task13['f']} + {task13['gcd']} * {task13['g']}')


НОД полиномов над GF(11): Poly(x, x, modulus=11)
f(x): Poly(5*x - 4, x, modulus=11)
g(x): Poly(3*x**5 + x**4 - 2*x**3 + x**2 + 4*x + 5, x, modulus=11)
gcd(f, g) = Poly(x, x, modulus=11) * Poly(5*x - 4, x, modulus=11) + Poly(x, x, modulus=11) * Poly(3*x**5 + x**4 - 2*x**3 + x**2 + 4*x + 5, x, modulus=11)


<h2>Task 14</h2>

Найти обратный элемент.

In [ ]:
def inverse_polynomial_mod_g(N: int) -> dict:
    x = symbols('x')

    s_coeffs = [(t + N) % 11 for t in range(11)]
    print(s_coeffs)
    
    f = Poly(s_coeffs[2] * x ** 2 + s_coeffs[1] * x + s_coeffs[0], x, domain=GF(13))
    g = Poly(x ** 8 + x ** 4 + x ** 3 + 6 * x + 2, x, domain=GF(13))

    inv = invert(f, g)
    return {'f': f, 'g': g, 'inverse': inv}


task14 = inverse_polynomial_mod_g(N)
print("Полином f(x):", task14['f'])
print("Полином g(x):", task14['g'])
print("Обратный элемент f^(-1) mod g:", task14['inverse'])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Полином f(x): Poly(2*x**2 + x, x, modulus=13)
Полином g(x): Poly(x**8 + x**4 + x**3 + 6*x + 2, x, modulus=13)
Обратный элемент f^(-1) mod g: Poly(4*x**7 + x**6 + 6*x**5 - 3*x**4 - x**3 + x**2 - 4*x, x, modulus=13)


<h2>Task 15</h2>

Реализуйте алгоритм генерации всех неприводимых полиномов степени d над простым конечным полем Fq, где q — простое. Протестируйте его для q = 2, 3, 5 и d = 2, 3, 4. 

In [152]:
def generate_irreducible_polynomials(q: int, d: int) -> list:
    """
    Возвращает список всех неприводимых полиномов степени d над F_q,
    где q — простое число.
    """

    x = symbols('x')
    field = GF(q)
    polys = []

    # Генерируем все полиномы степени d с коэффициентами из F_q
    for coeffs in product(range(q), repeat=d):
        # Пропускаем полиномы с нулевым старшим коэффициентом
        if coeffs[-1] == 0:
            continue

        # Формируем полином
        f = Poly(list(coeffs) + [1], x, domain=field)

        # Проверяем неприводимость
        if f.is_irreducible:
            polys.append(f)

    return polys


for q in [2, 3, 5]:
    for d in [2, 3, 4]:
        result = generate_irreducible_polynomials(q, d)
        print(f"\nПОЛЕ GF({q}), СТЕПЕНЬ {d}: найдено {len(result)} неприводимых полиномов")
        for p in result[:5]:
            print(" ", p)



ПОЛЕ GF(2), СТЕПЕНЬ 2: найдено 2 неприводимых полиномов
  Poly(x + 1, x, modulus=2)
  Poly(x**2 + x + 1, x, modulus=2)

ПОЛЕ GF(2), СТЕПЕНЬ 3: найдено 3 неприводимых полиномов
  Poly(x + 1, x, modulus=2)
  Poly(x**2 + x + 1, x, modulus=2)
  Poly(x**3 + x + 1, x, modulus=2)

ПОЛЕ GF(2), СТЕПЕНЬ 4: найдено 5 неприводимых полиномов
  Poly(x + 1, x, modulus=2)
  Poly(x**2 + x + 1, x, modulus=2)
  Poly(x**3 + x + 1, x, modulus=2)
  Poly(x**4 + x + 1, x, modulus=2)
  Poly(x**4 + x**3 + x**2 + x + 1, x, modulus=2)

ПОЛЕ GF(3), СТЕПЕНЬ 2: найдено 4 неприводимых полиномов
  Poly(x + 1, x, modulus=3)
  Poly(-x + 1, x, modulus=3)
  Poly(-x**2 + x + 1, x, modulus=3)
  Poly(-x**2 - x + 1, x, modulus=3)

ПОЛЕ GF(3), СТЕПЕНЬ 3: найдено 10 неприводимых полиномов
  Poly(x + 1, x, modulus=3)
  Poly(-x + 1, x, modulus=3)
  Poly(-x**2 + x + 1, x, modulus=3)
  Poly(-x**2 - x + 1, x, modulus=3)
  Poly(x**3 - x + 1, x, modulus=3)

ПОЛЕ GF(3), СТЕПЕНЬ 4: найдено 22 неприводимых полиномов
  Poly(x + 1, x, mod